In [1]:
import os
import sys
import git
import pathlib

import random
import time


import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist


PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

from libs.constants import model_seeds, error_seeds
from libs import utils
from libs.fitnessfns import ff_mnist32_cnn_ERR, eval_mnist32_cnn_ERR
from libs.ga.ga_utils import extract_best_worst_chromosomes

In [2]:
# set seed
this_seed = 84376
tf.random.set_seed(this_seed)
np.random.seed(this_seed)
random.seed(this_seed)
os.environ['PYTHONHASHSEED']=str(this_seed)

In [3]:
# Tensorflow Limit GPU usage
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        

In [4]:
# Prepare dataset
# Use only test images
DATASET_PATH = str(pathlib.Path(PROJ_ROOT_PATH / "datasets" / "mnist.npz" ))
(train_images, train_labels), (test_images, test_labels) = mnist.load_data(path=DATASET_PATH)
test_images = test_images.astype(np.float32) / 255.0  
test_images = np.expand_dims(test_images, axis=-1)
# resize the input shape , i.e. old shape: 28, new shape: 32
image_x_size = 32
image_y_size = 32
test_images = tf.image.resize(test_images, [image_x_size, image_y_size])

In [5]:
# Load test set
# Testing with only im_num images
im_num = 128*40#128*40=5120 #[128*78 = 9984]
test_set = (test_images[im_num:], 
            test_labels[im_num:])

In [6]:
model_seeds = model_seeds[0:3]
error_seeds = [error_seeds[0],error_seeds[2]]

In [7]:
error_profile_tag_list = []
ERR_PARAM_list = [2,3]
MAX_FAULT_PROB_LIST = [100E-3, 200E-3, 500E-3]
error_layer_list = ["h2", "op"]

for MAX_FAULT_PROB in MAX_FAULT_PROB_LIST:
    for seed in error_seeds:
        err_tag = "LIM_" + f"{int(MAX_FAULT_PROB*1E3):02d}" + "-" +str(seed)
        error_profile_tag_list.append(err_tag)

In [8]:
def eval_error_injected_with_shuffle(model, 
                                      layer_error_profiles, 
                                      ERR_PARAM,
                                      shuffle_orders,
                                      test_set):
    # intialize to None
    error_profile_c0=None
    error_profile_h0=None
    error_profile_h1=None
    error_profile_h2=None
    error_profile_op=None

    error_profile_c0 = layer_error_profiles["c0"]
    error_profile_h0 = layer_error_profiles["h0"]
    error_profile_h1 = layer_error_profiles["h1"]
    error_profile_h2 = layer_error_profiles["h2"]
    error_profile_op = layer_error_profiles["op"]

    
    # intialize to None
    shuffle_order_c0 = None
    shuffle_order_h0 = None
    shuffle_order_h1 = None
    shuffle_order_h2 = None
    shuffle_order_op = None
                    
    shuffle_order_c0 = shuffle_orders["c0"]
    shuffle_order_h0 = shuffle_orders["h0"]
    shuffle_order_h1 = shuffle_orders["h1"]
    shuffle_order_h2 = shuffle_orders["h2"]
    shuffle_order_op = shuffle_orders["op"]
        
    
    avg, std = eval_mnist32_cnn_ERR(model,
                                    error_profile_c0,
                                    error_profile_h0,
                                    error_profile_h1,
                                    error_profile_h2,
                                    error_profile_op,
                                    ERR_PARAM,
                                    shuffle_order_c0,
                                    shuffle_order_h0,
                                    shuffle_order_h1,
                                    shuffle_order_h2,
                                    shuffle_order_op,
                                    test_set)
        
    return avg

In [9]:
def reset_dict_values():
    dummy_dict =  {"c0":None,
             "h0":None,
             "h1":None,
             "h2":None,
             "op":None}
    return dummy_dict

In [10]:
dataset_seg_type = "all"
print("DATASET TYPE:\t", dataset_seg_type)

DATASET TYPE:	 all


In [11]:
model_type = "mnist32-cnn_1024_256_64"
results = {}
results_filename = 'evaluate_ga_genes-mnist32-cnn--h2_op--LIM_100_200_500-ERR_2_3.npy'

# one layer at a time
for model_seed in model_seeds:#model_seeds[0:2]:
    results[model_seed] = {}
    # Get model
    model_instance = model_type + "-" + str(model_seed)
    dataset, model_arch, model_config, layer_widths, seed = utils.instancename2metadata(model_instance)
    model_meta_type, model_type, model_instance = utils.metadata2instancenames(dataset, model_arch, layer_widths, seed)
    model_folder = pathlib.Path(PROJ_ROOT_PATH / "models" / model_type)
    model_filename = model_instance + ".h5"
    model_file = pathlib.Path(model_folder/ model_filename)
    # Load model
    model = tf.keras.models.load_model(model_file)
    print(model_instance)
    
    # Original Accuracy
    error_profiles = reset_dict_values()
    ERR_PARAM = None
    shuffle_orders = reset_dict_values()
    test_set = test_set
    original_accuracy = ff_mnist32_cnn_ERR(model,
                                            error_profile_c0=None,
                                            error_profile_h0=None,
                                            error_profile_h1=None,
                                            error_profile_h2=None,
                                            error_profile_op=None,
                                            ERR_PARAM=None,
                                            shuffle_order_c0=None,
                                            shuffle_order_h0=None,
                                            shuffle_order_h1=None,
                                            shuffle_order_h2=None,
                                            shuffle_order_op=None,
                                            test_set=test_set,
                                            batchsize=128).numpy()
    print("Original ACC: ", original_accuracy)
    results[model_seed]["original_acc"] = original_accuracy
    
    for error_profile_tag in error_profile_tag_list:
        print(error_profile_tag)
        results[model_seed][error_profile_tag] = {}
        # Load error profile
        error_profile_folder = pathlib.Path(PROJ_ROOT_PATH / "error_profiles")
        error_profile_filename = error_profile_tag + ".npy"
        error_profile_file = pathlib.Path(error_profile_folder/ error_profile_filename)
        error_profile = np.load(error_profile_file)

        for ERR_PARAM in ERR_PARAM_list:
            print("ERR_PARAM:\t", ERR_PARAM)
            results[model_seed][error_profile_tag][ERR_PARAM]={}
            for error_layer in error_layer_list:
                print("Layer: ", error_layer)
                results[model_seed][error_profile_tag][ERR_PARAM][error_layer]={}
                # Reset error profiles and shuffle orders for all layers
                error_profiles = reset_dict_values()
                shuffle_orders = reset_dict_values()
                
                # Inject error and observe degradation
                error_profiles[error_layer] = error_profile
                eval_acc  = eval_error_injected_with_shuffle(model, 
                                                              error_profiles, 
                                                              ERR_PARAM,
                                                              shuffle_orders,
                                                              test_set)
                results[model_seed][error_profile_tag][ERR_PARAM][error_layer]["faulty"] = eval_acc
                np.save(results_filename, results) 
                print("Faulty ACC: ", eval_acc)
                
                # Use the best chromosome and observe recovered value
                best, worst = extract_best_worst_chromosomes(dataset_seg_type, # (all, qmean1, qent2)
                                                                   error_layer, 
                                                                   ERR_PARAM, 
                                                                   model_instance, 
                                                                   error_profile_tag, 
                                                                   this_seed)
                # using best chromosome
                shuffle_orders[error_layer] = best
                eval_acc  = eval_error_injected_with_shuffle(model, 
                                                              error_profiles, 
                                                              ERR_PARAM,
                                                              shuffle_orders,
                                                              test_set)
                results[model_seed][error_profile_tag][ERR_PARAM][error_layer]["ga_optim"] = eval_acc
                np.save(results_filename, results) 
                print("GA ACC: ", eval_acc)
        print()
    print()

mnist32-cnn_1024_256_64-1023
Original ACC:  0.993421052631579
LIM_100-2188
ERR_PARAM:	 2
Layer:  h2
Faulty ACC:  0.990405701754386
GA ACC:  0.9906112938596491
Layer:  op
Faulty ACC:  0.9732044956140351
GA ACC:  0.9823876096491229
ERR_PARAM:	 3
Layer:  h2
Faulty ACC:  0.9897889254385964
GA ACC:  0.9903371710526315
Layer:  op
Faulty ACC:  0.9756716008771931
GA ACC:  0.981359649122807

LIM_100-4981
ERR_PARAM:	 2
Layer:  h2
Faulty ACC:  0.9899945175438596
GA ACC:  0.9905427631578948
Layer:  op
Faulty ACC:  0.9793722587719298
GA ACC:  0.9854029605263158
ERR_PARAM:	 3
Layer:  h2
Faulty ACC:  0.9902686403508772
GA ACC:  0.9906798245614036
Layer:  op
Faulty ACC:  0.9793722587719298
GA ACC:  0.9836211622807017

LIM_200-2188
ERR_PARAM:	 2
Layer:  h2
Faulty ACC:  0.9869791666666666
GA ACC:  0.9867735745614036
Layer:  op
Faulty ACC:  0.9536732456140351
GA ACC:  0.9672423245614036
ERR_PARAM:	 3
Layer:  h2
Faulty ACC:  0.986156798245614
GA ACC:  0.9884868421052632
Layer:  op
Faulty ACC:  0.951069078

In [12]:
# two layers together
for model_seed in model_seeds:#model_seeds[0:2]:
    # Get model
    model_instance = model_type + "-" + str(model_seed)
    dataset, model_arch, model_config, layer_widths, seed = utils.instancename2metadata(model_instance)
    model_meta_type, model_type, model_instance = utils.metadata2instancenames(dataset, model_arch, layer_widths, seed)
    model_folder = pathlib.Path(PROJ_ROOT_PATH / "models" / model_type)
    model_filename = model_instance + ".h5"
    model_file = pathlib.Path(model_folder/ model_filename)
    # Load model
    model = tf.keras.models.load_model(model_file)
    print(model_instance)
    
    for error_profile_tag in error_profile_tag_list:
        print(error_profile_tag)
        # Load error profile
        error_profile_folder = pathlib.Path(PROJ_ROOT_PATH / "error_profiles")
        error_profile_filename = error_profile_tag + ".npy"
        error_profile_file = pathlib.Path(error_profile_folder/ error_profile_filename)
        error_profile = np.load(error_profile_file)

        for ERR_PARAM in ERR_PARAM_list:
            print("ERR_PARAM:\t", ERR_PARAM)
            new_error_layer = "h2-op"
            print("Layer: ", new_error_layer)
            
            results[model_seed][error_profile_tag][ERR_PARAM][new_error_layer]={}
            # Reset error profiles and shuffle orders for all layers
            error_profiles = reset_dict_values()
            shuffle_orders = reset_dict_values()

            # Inject error in BOTH layers and observe degradation
            for error_layer in error_layer_list:
                error_profiles[error_layer] = error_profile

            eval_acc  = eval_error_injected_with_shuffle(model, 
                                                          error_profiles, 
                                                          ERR_PARAM,
                                                          shuffle_orders,
                                                          test_set)
            results[model_seed][error_profile_tag][ERR_PARAM][new_error_layer]["faulty"] = eval_acc
            np.save(results_filename, results) 
            print("Faulty ACC: ", eval_acc)

            # Get chromosomes for both layers
            for error_layer in error_layer_list:
                # Use the best chromosome and observe recovered value
                best, worst = extract_best_worst_chromosomes(dataset_seg_type, # (all, qmean1, qent2)
                                                                   error_layer, 
                                                                   ERR_PARAM, 
                                                                   model_instance, 
                                                                   error_profile_tag, 
                                                                   this_seed)
                # using best chromosome
                shuffle_orders[error_layer] = best
            eval_acc  = eval_error_injected_with_shuffle(model, 
                                                          error_profiles, 
                                                          ERR_PARAM,
                                                          shuffle_orders,
                                                          test_set)
            results[model_seed][error_profile_tag][ERR_PARAM][new_error_layer]["ga_optim"] = eval_acc
            np.save(results_filename, results) 
            print("GA ACC: ", eval_acc)
        print()
    print()

mnist32-cnn_1024_256_64-1023
LIM_100-2188
ERR_PARAM:	 2
Layer:  h2-op
Faulty ACC:  0.9689555921052632
GA ACC:  0.9771107456140351
ERR_PARAM:	 3
Layer:  h2-op
Faulty ACC:  0.9679276315789475
GA ACC:  0.9760142543859649

LIM_100-4981
ERR_PARAM:	 2
Layer:  h2-op
Faulty ACC:  0.9736156798245613
GA ACC:  0.9808799342105262
ERR_PARAM:	 3
Layer:  h2-op
Faulty ACC:  0.9746436403508772
GA ACC:  0.9784128289473685

LIM_200-2188
ERR_PARAM:	 2
Layer:  h2-op
Faulty ACC:  0.9385279605263158
GA ACC:  0.9533991228070176
ERR_PARAM:	 3
Layer:  h2-op
Faulty ACC:  0.9347587719298246
GA ACC:  0.9524396929824562

LIM_200-4981
ERR_PARAM:	 2
Layer:  h2-op
Faulty ACC:  0.9492872807017544
GA ACC:  0.9613486842105262
ERR_PARAM:	 3
Layer:  h2-op
Faulty ACC:  0.9507264254385964
GA ACC:  0.9578536184210527

LIM_500-2188
ERR_PARAM:	 2
Layer:  h2-op
Faulty ACC:  0.8114035087719298
GA ACC:  0.8527275219298246
ERR_PARAM:	 3
Layer:  h2-op
Faulty ACC:  0.810718201754386
GA ACC:  0.8580043859649122

LIM_500-4981
ERR_PARAM